# Python notebook for analyzing AirBnB dataset from I2I #

### Importing modules ###

In [63]:
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)
# iplot does not work for now so no inline plotting

### Reading in data ###

The price per m2 is from: https://maps.amsterdam.nl/woningwaarde/

In [64]:
def average_m2_prices(df, labels):
    df_new = df.copy()
    for i, label in enumerate(labels):
        label_list = str.split(label)
        if len(label_list) == 3:
            a, _, b = label_list
            df_new.loc[df_new.LABEL == label, 'LABEL'] = (int(a)+int(b))/2
    return df_new

In [84]:
df = pd.read_csv('listings_with_prices.csv')
df = df.drop(['name', 'host_name', 'neighbourhood_group', 'latitude', 'longitude', 'Lower', 'Upper', 'SELECTIE', 'FILTER'], axis=1)
# There is a very high outlier with a price of 8616 while the next lowest is 1442
df = df[df.price != 8616]
df = df.sort_values(by=['LABEL'])



ranges = df.LABEL.unique()
ranges = [r for r in ranges if str(r) != 'nan']

df = average_m2_prices(df, ranges)
df['yearly_returns'] = df.price * 365
df['monthly_returns'] = df.yearly_returns / 12

df.head()

,id,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,LABEL,yearly_returns,monthly_returns
3181,5362748,27800005,Bijlmer-Oost,Private room,45,1,8,2016-06-29,"0,68",1,193,2207.5,16425,1368.750000
3236,3651182,17092248,Bijlmer-Oost,Entire home/apt,100,2,40,2016-06-28,"1,92",1,347,2207.5,36500,3041.666667
3235,12272272,12308050,Bijlmer-Oost,Private room,65,3,7,2016-06-10,"2,69",2,355,2207.5,23725,1977.083333
3234,12735030,52920463,Bijlmer-Oost,Entire home/apt,45,3,1,2016-06-28,1,1,6,2207.5,16425,1368.750000
3233,12680964,56894783,Bijlmer-Oost,Private room,30,1,3,2016-06-27,"1,48",1,0,2207.5,10950,912.500000


## Analyzing financial stuff ##

In order to anylyze the Return On Investment (ROI) we need two measures:

1) the Investment. We have the m^2 price, however we do not know how big the apparments are. Also we have to include other costs like elektricity and gas. 

2) the Return. We have the price for 1 night, we assume that the houses are rented permanentely.

In [23]:
labels = df.LABEL.unique()
values = [len(df[df.LABEL == label])  for label in labels]
    
trace1 = go.Bar(
    x=ranges, 
    y=values, 
    name='Boxplot of location cost')

layout = go.Layout(
    title='Locations costs', 
    xaxis=dict(title='location cost'), 
    yaxis=dict(title='frequency'))

data = [trace1]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='barplot_cost.html')

'file://C:\\Users\\Johan van den Heuvel\\Documents\\AirBnB\\barplot_cost.html'

We want to see if we can group the prices by the labels, i.e. how much they cost. 

Seems to work but need statistical test to make sure it is a significant difference.

In [5]:
df_grouped_by_label = [df.loc[df.LABEL == label] for label in labels]

for i, label in enumerate(labels):
    df_temp = df_grouped_by_label[i]
    
    if isinstance(label, float):
        y = df_temp.price
        mean = np.full(len(df_temp), np.mean(df_temp.price))
        x = np.arange(len(df_temp))

        trace = go.Box(
            y=y,
            name = 'Box Plot',
            boxpoints='all',
            jitter=0.3
        )

        layout = go.Layout(
            title='prices for {}'.format(label), 
            xaxis=dict(title='id'), 
            yaxis=dict(title='price'))

        data = [trace]
        figure = go.Figure(data=data, layout=layout)

        plot(figure, filename='price_{}.html'.format(label))

Above we added the avg price per m2 for houses in the corresponding area's. Now we need some measure for the expected size of the appartment.

Expected size of house are based on numbers from CBS: https://www.cbs.nl/nl-nl/nieuws/2018/22/amsterdamse-huishoudens-hebben-minder-woonoppervlakte

In [6]:
for i, label in enumerate(labels):
    df_temp = df_grouped_by_label[i]
    
    if isinstance(label, float):
        ROI = df_temp.yearly_returns / (df_temp.LABEL * 50)
        # TODO figure out the correct m2 size
        # TODO add other monthly costs like elektrictity
        ROI = ROI.sort_values()
        ROI = ROI.reset_index(drop=True)
        
        ROI_2 = df_temp.yearly_returns / (df_temp.LABEL * 50 + 200*12)
        ROI_2 = ROI_2.sort_values()
        ROI_2 = ROI_2.reset_index(drop=True)

        trace1 = go.Scatter(
            y=ROI,
            x=ROI.index,
            name = 'ROI plot',
            mode = 'markers'
        )
        
        trace2 = go.Scatter(
            y=ROI_2,
            x=ROI.index,
            name = 'ROI-2 plot',
            mode = 'markers'
        )

        layout = go.Layout(
            title='ROI for {}'.format(label), 
            xaxis=dict(title='id'), 
            yaxis=dict(title='price'))

        data = [trace1]
        figure = go.Figure(data=data, layout=layout)

        plot(figure, filename='ROI_{}.html'.format(label))

## Analyzing different groups (room types) ##

In [76]:
groups = df.room_type.unique()
values = [len(df[df.room_type == group])  for group in groups]

print(groups)


trace1 = go.Bar(
    x=groups, 
    y=values, 
    name='Barplot of location types')

layout = go.Layout(
    title='Location types', 
    xaxis=dict(title='locations'), 
    yaxis=dict(title='frequency'))

data = [trace1]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='Location types.html')

['Shared room' 'Private room' 'Entire home/apt']


'file://C:\\Users\\Johan van den Heuvel\\Documents\\AirBnB\\Location types.html'

In [94]:
df_privateroom = df.loc[df.room_type == 'Private room']

maximum = 972

x = np.arange(len(df_privateroom))
y = df_privateroom.monthly_returns

trace1 = go.Scatter(
    x=x, 
    y=y, 
    name='private room prices',
    mode='markers')

# https://www.huurcommissie.nl/fileadmin/afbeeldingen/Downloads/Huurprijstabellen/Maximale_huurprijsgrenzen_voor_onzelfstandige_woonruimten_per_1_juli_2016_01.pdf
y = np.full(len(df_privateroom),maximum)

trace2 = go.Scatter(
    x=x, 
    y=y, 
    name='maximum price for 750 points')

layout = go.Layout(
    title='private room prices and maximum price', 
    xaxis=dict(title='id'), 
    yaxis=dict(title='monthly price'))

data = [trace1, trace2]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='private room prices and maximum price.html')

print(sum(df_privateroom.monthly_returns - maximum))

3741747.8333333246


In [95]:
df_entirehome = df.loc[df.room_type == 'Entire home/apt']

maximum = 1250

x = np.arange(len(df_entirehome))
y = df_entirehome.monthly_returns

# https://www.huurcommissie.nl/fileadmin/afbeeldingen/Downloads/Huurprijstabellen/Maximale_huurprijsgrenzen_voor_zelfstandige_woonruimten_per_1_juli_2016_01.pdf
trace1 = go.Scatter(
    x=x, 
    y=y, 
    name='full home prices',
    mode='markers')

# https://www.huurcommissie.nl/fileadmin/afbeeldingen/Downloads/Huurprijstabellen/Maximale_huurprijsgrenzen_voor_onzelfstandige_woonruimten_per_1_juli_2016_01.pdf
y = np.full(len(df_entirehome),maximum)

trace2 = go.Scatter(
    x=x, 
    y=y, 
    name='maximum price for 250 points')

layout = go.Layout(
    title='full home prices and maximum price', 
    xaxis=dict(title='id'), 
    yaxis=dict(title='monthly price'))

data = [trace1, trace2]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='full home prices and maximum price.html')

print(sum(df_entirehome.monthly_returns - maximum))

35312931.66666673


## Analyzing different groups (neighbourhood) ##

In [96]:
neighbourhoods = df.neighbourhood.unique()
values = [len(df[df.neighbourhood == neighbourhood])  for neighbourhood in neighbourhoods]

print(neighbourhoods)

trace1 = go.Bar(
    x=neighbourhoods, 
    y=values, 
    name='Barplot of neighbourhoods')

layout = go.Layout(
    title='Neighbourhoods', 
    xaxis=dict(title='neighbourhoods'), 
    yaxis=dict(title='frequency'))

data = [trace1]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='Neighbourhoods.html')

['Bijlmer-Oost' 'Gaasperdam - Driemond' 'Bijlmer-Centrum'
 'Geuzenveld - Slotermeer' 'Noord-West' 'De Aker - Nieuw Sloten' 'Osdorp'
 'Bos en Lommer' 'Noord-Oost' 'Slotervaart' 'IJburg - Zeeburgereiland'
 'Watergraafsmeer' 'Buitenveldert - Zuidas' 'De Pijp - Rivierenbuurt'
 'Oud-Noord' 'Oostelijk Havengebied - Indische Buurt' 'Centrum-Oost'
 'Centrum-West' 'Oud-Oost' 'De Baarsjes - Oud-West' 'Westerpark' 'Zuid']


'file://C:\\Users\\Johan van den Heuvel\\Documents\\AirBnB\\Neighbourhoods.html'

In [ ]:
neighbourhoods = df.neighbourhood.unique()
df_grouped_by_neighbourhoods = [df.loc[df.neighbourhood == neighbourhood] for neighbourhood in neighbourhoods]

for i, neighbourhood in enumerate(neighbourhoods):
    df_temp = df_grouped_by_neighbourhoods[i]
    
    if isinstance(label, float):
        y = df_temp.price
        mean = np.full(len(df_temp), np.mean(df_temp.price))
        x = np.arange(len(df_temp))

        trace = go.Box(
            y=y,
            name = 'Box Plot',
            boxpoints='all',
            jitter=0.3
        )

        layout = go.Layout(
            title='prices for {}'.format(label), 
            xaxis=dict(title='id'), 
            yaxis=dict(title='price'))

        data = [trace]
        figure = go.Figure(data=data, layout=layout)

        plot(figure, filename='price_{}.html'.format(label))

## Rules and Regulations ##

There are new rules since 2017, however because the dataset is from 2016 these do not apply. These rules state that you can only rent for 60 days a year, in 2019 it went down to 30 days a year. Also since 1th of Oktober 2017 you have to report when renters come. 

To figure out how many points a house has is not really doable with this data. The amount of uncertainty would be huge.

In conclusion, I cannot really find rules and regulations to check.

## Analyzing amount of days rented ##

Using the reviews, minimum nights, and availability we might be able to make a guess how much a certain house is rented for per year.

In [8]:
df.head()

,id,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,LABEL,yearly_returns
3181,5362748,27800005,Bijlmer-Oost,Private room,45,1,8,2016-06-29,"0,68",1,193,2207.5,16425
3236,3651182,17092248,Bijlmer-Oost,Entire home/apt,100,2,40,2016-06-28,"1,92",1,347,2207.5,36500
3235,12272272,12308050,Bijlmer-Oost,Private room,65,3,7,2016-06-10,"2,69",2,355,2207.5,23725
3234,12735030,52920463,Bijlmer-Oost,Entire home/apt,45,3,1,2016-06-28,1,1,6,2207.5,16425
3233,12680964,56894783,Bijlmer-Oost,Private room,30,1,3,2016-06-27,"1,48",1,0,2207.5,10950


In [9]:
df_days_rented = df.drop(['id', 'host_id', 'yearly_returns'], axis=1)
df_days_rented.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,LABEL
3181,Bijlmer-Oost,Private room,45,1,8,2016-06-29,"0,68",1,193,2207.5
3236,Bijlmer-Oost,Entire home/apt,100,2,40,2016-06-28,"1,92",1,347,2207.5
3235,Bijlmer-Oost,Private room,65,3,7,2016-06-10,"2,69",2,355,2207.5
3234,Bijlmer-Oost,Entire home/apt,45,3,1,2016-06-28,1,1,6,2207.5
3233,Bijlmer-Oost,Private room,30,1,3,2016-06-27,"1,48",1,0,2207.5


In [67]:
labels = df.reviews_per_month
values = df.price

df = df.sort_values(by=['reviews_per_month'])

trace1 = go.Bar(
    x=values, 
    y=labels, 
    name='Boxplot of location cost')

layout = go.Layout(
    title='Reviews vs Price', 
    yaxis=dict(title='reviews per month'), 
    xaxis=dict(title='price per night'))

data = [trace1]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='ReviewsPrice.html')

'file://C:\\Users\\Johan van den Heuvel\\Documents\\AirBnB\\ReviewsPrice.html'

In [72]:
reviews = df.reviews_per_month.str.replace(',', '.')
reviews = reviews.astype(float)

labels = df.price * reviews
# labels = labels.sort_values()
labels = labels.fillna(0)
values = df.price

df = df.sort_values(by=['price'])

trace1 = go.Bar(
    x=values, 
    y=labels, 
    name='Boxplot of location cost'
)

layout = go.Layout(
    title='Reviews vs Price', 
    yaxis=dict(title='price / reviews_per_month'), 
    xaxis=dict(title='price per night'))

data = [trace1]
figure = go.Figure(data=data, layout=layout)

plot(figure, filename='ReviewsPrice.html')

'file://C:\\Users\\Johan van den Heuvel\\Documents\\AirBnB\\ReviewsPrice.html'